# Communicating with the RBD 9103 Picoammeter in Python

#### Version 1 -  18/08/2015

## Getting Set Up

Please read this in consultation with the user manual.

The picoammeter can be accessed as a serial device on the PC by reading/writing text commands, or by using the *Actual* software that comes with it. The *Actuel* software includes a console that can be used to write and read commands. Alternatively I used the *termite* (http://www.compuphase.com/software_termite.htm) for testing.

to use in termite or Python we must first find out what COM port it is on. Do this by looking for *FT232R USB UART* in Windows (7) *Devices and Printers*, double click on it and select *Hardware* and we see the COM port (COM3 when I tested this one). 

## Configuration and Essential Commands

To establish serial communications with the device we need to have the right settings, from the manual (*Communications and Command Reference*):

| Setting | Value |
|---------|-------|
| Bits per Second (Baud Rate) | 57600 |
| Data Bits | 8 |
| Parity | None |
| Stop Bits | 1 |
| Flow Control | None | 

The most useful commands for talking to the device are:

| Command | Use |
|---------|-----|
| &Q      | Query Current Configuration (returns multiple lines) | 
| &RX     | Set range, 0 is auto range, for others see manual |
| &FXXX   | Sets the filter time constant - maybe leave at default|
| &S      | Reguest a sample |
| &VX     | Set number of digits (5, 6, 7 or 8), Default is 5 |

For other commands see the manual.

## Using Python

### PySerial

To communicate with a serial device in Python we wil use PySerial. If not installed install with 

    conda install pyserial 
    
The code below shows how to import pyserial and set up a connection. It is essential to set the timeout as well as otherwise if you try to read the device and it has no data it will block indefinitely.

In [1]:
import serial
ser=serial.Serial('com3',
                 baudrate=57600,
                 bytesize=serial.EIGHTBITS,
                 parity=serial.PARITY_NONE,
                 stopbits=serial.STOPBITS_ONE,
                 xonxoff=False,
                 timeout=1)

### Strings and byte arrays

Arrays of bytes are sent to and read from the device. Thus strings must be converted to bytes and vice-versa. To convert a string to bytes use:

    bytes(string_name,'utf-8') 

where 'utf-8' is a common string encoding.

Similarly to convert bytes to a string use:

    .decode('utf-8')

on the bytes array. Examples below will make clearer.

Useful helper function:

In [10]:
def data(mystring):
    return bytes(mystring,'utf-8')

### Writing to and reading from the device

Write a command with the PySerial <code> write() </code> command.
Note: ** It is essential to end every write to the device with a newline **

After each write use <code> readline() </code> or <code> readlines() </code> to read back the decive's response, even if you are just setting some parameter (like the range). 

Notes:

<code> readline() </code> Read a line which is terminated with end-of-line (eol) character (\n by default) or until timeout.

<code> readlines() </code> Read a list of lines, until timeout. sizehint is ignored and only present for API compatibility with built-in File objects. Note that this function only returns on a timeout.

The byte arrays/strings returned are terminated by carriage returns (\r) and newlines (\n)

### Examples:

In [11]:
# Example: get current configuration
ser.write(data('&Q\n'))
ser.readlines()

[b'RBD Instruments: PicoAmmeter\r\n',
 b'Firmware Version: 01.37\r\n',
 b'Build: 5-14-13\r\n',
 b'R, Range=AutoR\r\n',
 b'I, sample Interval=0000 mSec\r\n',
 b'L, Chart Log Update Interval=0200 mSec\r\n',
 b'B, BIAS=OFF\r\n',
 b'F, Filter=032\r\n',
 b'V, FormatLen=8\r\n',
 b'CA, Autocal=OFF\r\n',
 b'G, AutoGrounding=DISABLED\r\n',
 b'Q, State=MEASURE\r\n',
 b'P, PID=Pico1     \r\n']

In [12]:
# Example: put in autorange

ser.write(data('&R0\n')) # put in autorange
ser.readline()

b'&R, Range=AutoR\r\n'

In [13]:
# Example: Set number of Digits

ser.write(data('&V8\n')) # 8 decimal places
ser.readline()

b'&V, FormatLen=8\r\n'

### Reading current samples.

When the <code> &S </code> is sent the device returns a byte array that contains information (comma separated) on the range, the current and the scale. This byte array will have to be converted to a string, have the trailing \r and \n removed, split and then converted into an appropriate numberic value.

The allowed ranges are 'nA, 'uA', 'mA', so the range will have to be checked and the numeric value multiplied by the appropriate scale factor.

The first part of the returned command is usually '&S=' but unstable values are reported with an asterix, e,g.

    &S*,Range=002mA,+0.0009572,mA 
The manual states that after changing the range it may take a few readings before the readings are stable and accurate.

Also, over-range status is indicated by a '>' and under-range status by a '<' after the '&S', so one should always check the first character after the '&S' to determine the trustworthyness of the results, and perhaps re-read or change the scale (if not on autoranging).

In [16]:
# Example: request a sample:

ser.write(data('&S\n'))
ans=ser.readline().decode('utf-8').rstrip() # convert bytes to string and remove trailing newline and CR
print(ans)

&S=,Range=002nA,+0.0023719,nA


In [17]:
ans.split(',')

['&S=', 'Range=002nA', '+0.0023719', 'nA']